# Final Project

## Load Data

In [9]:
import yfinance as yf
from pypfopt import plotting, EfficientFrontier, objective_functions, expected_returns
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from pypfopt import BlackLittermanModel, black_litterman, risk_models


tickers = ["AAPL", "AMZN", "BRK-B", "GOOGL", "META", "MSFT", "NVDA",
           "TSLA", "UNH", "XOM", "JPM", "JNJ", "V", "LLY", "PG"]
portfolio_data = yf.download(tickers, period="5y")
portfolio_data = portfolio_data["Adj Close"]
portfolio_data.tail()

[*********************100%***********************]  15 of 15 completed


,AAPL,AMZN,BRK-B,GOOGL,JNJ,JPM,LLY,META,MSFT,NVDA,PG,TSLA,UNH,V,XOM
Date,,,,,,,,,,,,,,,
2023-06-27,188.059998,129.179993,335.339996,118.330002,163.289993,139.199997,464.500000,287.049988,334.570007,418.760010,150.020004,250.210007,482.559998,227.339996,104.550003
2023-06-28,189.250000,129.039993,334.149994,120.180000,162.960007,138.589996,458.700012,285.290009,335.850006,411.170013,149.990005,256.239990,474.450012,227.960007,105.400002
2023-06-29,189.589996,127.900002,336.910004,119.099998,164.100006,143.429993,464.489990,281.529999,335.049988,408.220001,149.380005,257.500000,476.440002,234.320007,106.699997
2023-06-30,193.970001,130.360001,341.000000,119.699997,165.520004,145.440002,468.980011,286.980011,340.540009,423.019989,151.740005,261.769989,480.640015,237.479996,107.250000
2023-07-03,192.460007,130.220001,342.000000,119.900002,163.350006,146.610001,461.420013,286.019989,337.989990,424.130005,152.500000,279.820007,477.880005,237.970001,107.459999


In [10]:
market_index_ticker = "^GSPC"  # S&P 500 index
market_index_data = yf.download(market_index_ticker, period="5y")
market_index_data = market_index_data["Adj Close"]
market_index_data.tail()

[*********************100%***********************]  1 of 1 completed


Date
2023-06-27    4378.410156
2023-06-28    4376.859863
2023-06-29    4396.439941
2023-06-30    4450.379883
2023-07-03    4455.589844
Name: Adj Close, dtype: float64

In [11]:
# Utility function to predict stock price for 10 future days using a non-linear regression model
def forecast_price(data, reg_model):
    dev_x = [data[i:i+10] for i in range(len(data)-20)]
    dev_y = [data[i+10] for i in range(10,len(data)-10)]
    test = [data[i:i+10] for i in range(len(data)-20,len(data)-10)]
    reg_model.fit(dev_x,dev_y)
    pred = reg_model.predict(test)
    return pred

## Mean Variance Optimization

In [12]:
reg_models = [RandomForestRegressor(), GradientBoostingRegressor(), MLPRegressor(hidden_layer_sizes=(100, 100)), SVR()]

for reg_model in reg_models:
    print("----------------------------------------------------")
    print(f"Non-linear ML model: {reg_model}\n")
    
    future_prices = {}
    for ticker in tickers:
        future_prices[ticker] = forecast_price(portfolio_data[ticker], reg_model)

    future_prices = pd.DataFrame(future_prices)
    S = risk_models.CovarianceShrinkage(future_prices).ledoit_wolf()

    mu = expected_returns.capm_return(future_prices)

    ef = EfficientFrontier(mu, S)
    ef.min_volatility()
    weights = ef.clean_weights()
    print(f"Weights: {weights}")

    ret, volatility, sharpe_ratio = ef.portfolio_performance()
    print("Expected annual return:", ret)
    print("Annual volatility:", volatility)
    print("Sharpe ratio:", sharpe_ratio)

----------------------------------------------------
Non-linear ML model: RandomForestRegressor()

Weights: OrderedDict([('AAPL', 0.09829), ('AMZN', 0.0), ('BRK-B', 0.08532), ('GOOGL', 0.03853), ('META', 0.05345), ('MSFT', 0.09542), ('NVDA', 0.09795), ('TSLA', 0.0), ('UNH', 0.09128), ('XOM', 0.09379), ('JPM', 0.02113), ('JNJ', 0.08051), ('V', 0.07559), ('LLY', 0.09787), ('PG', 0.07088)])
Expected annual return: 0.18905895581684629
Annual volatility: 0.0918826594892591
Sharpe ratio: 1.8399440847335178
----------------------------------------------------
Non-linear ML model: GradientBoostingRegressor()

Weights: OrderedDict([('AAPL', 0.10738), ('AMZN', 0.0), ('BRK-B', 0.09247), ('GOOGL', 0.04598), ('META', 0.08368), ('MSFT', 0.03926), ('NVDA', 0.10203), ('TSLA', 0.0), ('UNH', 0.10583), ('XOM', 0.08291), ('JPM', 0.01742), ('JNJ', 0.0771), ('V', 0.05106), ('LLY', 0.10563), ('PG', 0.08925)])
Expected annual return: 0.12073700058632124
Annual volatility: 0.057315238427784604
Sharpe ratio: 1.

/opt/homebrew/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Weights: OrderedDict([('AAPL', 0.00079), ('AMZN', 0.04963), ('BRK-B', 0.13639), ('GOOGL', 0.12173), ('META', 0.0), ('MSFT', 0.08581), ('NVDA', 0.0), ('TSLA', 0.0), ('UNH', 0.03846), ('XOM', 0.07404), ('JPM', 0.04086), ('JNJ', 0.14674), ('V', 0.05405), ('LLY', 0.06787), ('PG', 0.18364)])
Expected annual return: 0.08323087872019111
Annual volatility: 0.04081713595254665
Sharpe ratio: 1.5491258081826789
----------------------------------------------------
Non-linear ML model: SVR()

Weights: OrderedDict([('AAPL', 0.14493), ('AMZN', 0.02331), ('BRK-B', 0.09521), ('GOOGL', 0.013), ('META', 0.12612), ('MSFT', 0.04146), ('NVDA', 0.01814), ('TSLA', 0.01142), ('UNH', 0.06791), ('XOM', 0.10119), ('JPM', 0.00181), ('JNJ', 0.04661), ('V', 0.0507), ('LLY', 0.16418), ('PG', 0.09398)])
Expected annual return: 0.001021012153627639
Annual volatility: 0.009272883646401407
Sharpe ratio: -2.0467190757578058


## Index model

In [13]:
reg_models = [RandomForestRegressor(), GradientBoostingRegressor(), MLPRegressor(hidden_layer_sizes=(100, 100)), SVR()]

for reg_model in reg_models:
    print("----------------------------------------------------")
    print(f"Non-linear ML model: {reg_model}\n")

    returns = {"MARKET": ((market_index_data / market_index_data.shift(1)) - 1).dropna().tolist()}
    for ticker in tickers:
        returns[ticker] = ((portfolio_data[ticker] / portfolio_data[ticker].shift(1))-1).dropna().tolist()
    market_future = forecast_price(returns["MARKET"], reg_model)


    beta = {}
    alpha = {}
    residual_variance = {}
    for ticker in tickers:
        stock_future = forecast_price(returns[ticker], reg_model)

        single_index_reg = LinearRegression()
        single_index_reg.fit(np.array(market_future).reshape(-1, 1), y=stock_future)

        beta[ticker] = single_index_reg.coef_[0]
        alpha[ticker] = single_index_reg.intercept_

        y_pred = single_index_reg.predict(np.array(returns["MARKET"]).reshape(-1, 1))
        residuals = returns[ticker] - y_pred
        residual_variance[ticker] = np.var(residuals)


    # Compute the initial position of each security
    weights = {ticker: alpha[ticker] / residual_variance[ticker] for ticker in tickers}
    total_weight = sum(weights.values())
    weights = {ticker: weight / total_weight for ticker, weight in weights.items()}

    # Compute the alpha of the active portfolio
    alpha_portfolio = sum(weights[ticker] * alpha[ticker] for ticker in tickers)

    # Compute the residual variance of the active portfolio
    residual_variance_portfolio = sum((weights[ticker] ** 2) * residual_variance[ticker] for ticker in tickers)

    # Compute the initial position in the active portfolio
    residual_variance_market = 0.0114  # Variance of S&P 500
    risk_premium_market = 0.056
    initial_position_portfolio = (alpha_portfolio * residual_variance_market) / (residual_variance_portfolio * risk_premium_market)

    # Compute the beta of the active portfolio
    beta_portfolio = sum(weights[ticker] * beta[ticker] for ticker in tickers)

    # Adjust the initial position in the active portfolio
    adjusted_position_portfolio = initial_position_portfolio / (1 + (1 - beta_portfolio) * initial_position_portfolio)

    # Optimal risky portfolio now has weights
    final_weight_market = 1 - adjusted_position_portfolio
    weights = {ticker: weight * adjusted_position_portfolio for ticker, weight in weights.items()}

    # Calculate the risk premium of the portfolio
    risk_premium_portfolio = (final_weight_market + adjusted_position_portfolio * beta_portfolio) * risk_premium_market + adjusted_position_portfolio * alpha_portfolio

    # Compute the variance of the portfolio
    portfolio_variance = (final_weight_market + adjusted_position_portfolio * beta_portfolio) ** 2 * residual_variance_market + adjusted_position_portfolio ** 2 * residual_variance_portfolio

    # Calculate the Sharpe ratio
    sharpe_ratio = risk_premium_portfolio / (portfolio_variance ** 0.5)

    print(f"Weights: {weights}")
    print(f"Market weight: {final_weight_market}")
    print(f"Sharpe ratio: {sharpe_ratio}")

----------------------------------------------------
Non-linear ML model: RandomForestRegressor()

Weights: {'AAPL': 0.10453201485696424, 'AMZN': -0.13258494822887473, 'BRK-B': -0.1963424444239801, 'GOOGL': 0.09186390491669533, 'META': 0.013003633795059514, 'MSFT': 0.00031199712072986214, 'NVDA': 0.06377117126433944, 'TSLA': -0.008407295867087704, 'UNH': -0.0846079077907419, 'XOM': -0.009685539513443914, 'JPM': 0.039675946708028605, 'JNJ': -0.07465100959610031, 'V': 0.047139111449644555, 'LLY': -0.006805069550490406, 'PG': 0.017913851537555578}
Market weight: 1.134872583321702
Sharpe ratio: 0.5725357588046383
----------------------------------------------------
Non-linear ML model: GradientBoostingRegressor()

Weights: {'AAPL': 0.5893596190773872, 'AMZN': -0.41943281389669523, 'BRK-B': -0.31949436738129106, 'GOOGL': 0.1025019217075352, 'META': -0.08398804704080262, 'MSFT': 0.662550995084664, 'NVDA': 0.3508662963007942, 'TSLA': 0.16029632268070615, 'UNH': 0.008976857885040573, 'XOM': 0.

## Capital asset pricing model (CAPM)

In [14]:
reg_models = [RandomForestRegressor(), GradientBoostingRegressor(), MLPRegressor(hidden_layer_sizes=(100, 100)), SVR()]

for reg_model in reg_models:
    print("----------------------------------------------------")
    print(f"Non-linear ML model: {reg_model}")
    
    returns = {"MARKET": ((market_index_data / market_index_data.shift(1)) - 1).dropna().tolist()}
    for ticker in tickers:
        returns[ticker] = ((portfolio_data[ticker] / portfolio_data[ticker].shift(1))-1).dropna().tolist()
    market_future = forecast_price(returns["MARKET"], reg_model)

    future_returns = {}
    for ticker in tickers:
        single_index_reg = Ridge()
        single_index_reg.fit(np.array(returns["MARKET"]).reshape(-1, 1), y=returns[ticker])
        future_returns[ticker] = single_index_reg.predict(np.array(market_future).reshape(-1, 1))

    future_returns = pd.DataFrame(future_returns)

    S = risk_models.CovarianceShrinkage(future_returns).ledoit_wolf()
    ef = EfficientFrontier(None, S)
    ef.min_volatility()
    weights = ef.clean_weights()
    print(weights)

    ret, volatility, sharpe_ratio = ef.portfolio_performance()
    print("Expected annual return:", ret)
    print("Annual volatility:", volatility)
    print("Sharpe ratio:", sharpe_ratio)

----------------------------------------------------
Non-linear ML model: RandomForestRegressor()
OrderedDict([('AAPL', 0.12686), ('AMZN', 0.0), ('BRK-B', 0.0), ('GOOGL', 0.00382), ('META', 0.0), ('MSFT', 0.11099), ('NVDA', 0.13603), ('TSLA', 0.17044), ('UNH', 0.07531), ('XOM', 0.02288), ('JPM', 0.0), ('JNJ', 0.03719), ('V', 0.0), ('LLY', 0.17838), ('PG', 0.13811)])
Expected annual return: None
Annual volatility: 8.162879437618876
Sharpe ratio: None
----------------------------------------------------
Non-linear ML model: GradientBoostingRegressor()
OrderedDict([('AAPL', 0.06013), ('AMZN', 0.0), ('BRK-B', 0.0), ('GOOGL', 0.0), ('META', 0.0), ('MSFT', 0.0), ('NVDA', 0.11199), ('TSLA', 0.32503), ('UNH', 0.0), ('XOM', 0.0), ('JPM', 0.0), ('JNJ', 0.0), ('V', 0.0), ('LLY', 0.37879), ('PG', 0.12404)])
Expected annual return: None
Annual volatility: 3.976388775823544
Sharpe ratio: None
----------------------------------------------------
Non-linear ML model: MLPRegressor(hidden_layer_sizes=(1

## Arbitrage pricing theory and multifactor model

In [31]:
reg_models = [RandomForestRegressor(), GradientBoostingRegressor(), MLPRegressor(hidden_layer_sizes=(100, 100)), SVR()]

for reg in reg_models:
    print("----------------------------------------------------")
    print(f"Non-linear ML model: {reg}")
    
    # Fetch historical data for the macroeconomic factors
    factor_tickers = ['SPY', 'TLT', 'BND']
    factor_data = yf.download(factor_tickers, period="5y")
    factor_data = factor_data['Adj Close']

    # Calculate the returns for the S&P and Treasury bond factors
    factor_returns = factor_data.pct_change().dropna()
    spy_returns = factor_returns['SPY']
    tlt_returns = factor_returns['TLT']
    bnd_returns = factor_returns['BND']

    # FACTOR 1
    bnd_future = forecast_price(bnd_returns, reg_model)

    # FACTOR 2
    spy_future = forecast_price(spy_returns, reg_model)

    # FACTOR 3
    tlt_future = forecast_price(tlt_returns, reg_model)

    # FACTOR 4
    # GDP Price of USA for past 5 years

    # US GDP per capita Prices of last 5 years
    GDP_prices = {
        2018: 59607,
        2019: 60698,
        2020: 58453,
        2021: 61855,
        2022: 62551,
        2023: 63451 # Forecast data also available online
    }

    # Normalize the GDP values as it will make the other factors irrelevant as it is very large
    values = list(GDP_prices.values())
    values_array = [[value] for value in values]
    scaler = StandardScaler()
    scaled_values = scaler.fit_transform(values_array)
    scaled_values = scaled_values.flatten()
    scaled_GDP_prices = {year: scaled_value for year, scaled_value in zip(GDP_prices.keys(), scaled_values)}

    # print(scaled_GDP_prices)

    for index, row in factor_returns.iterrows():
        # Extract the year from the date
        year = index.year

        # Fill the 'GDP' column with the corresponding GDP price based on the year
        factor_returns.at[index, 'GDP'] = scaled_GDP_prices.get(year)

    future_factors = {'BND': bnd_future, 'SPY': spy_future, 'TLT': tlt_future, 'GDP': [scaled_GDP_prices.get(2023)]*10}
    future_factors = pd.DataFrame(future_factors)

    returns = {}
    future_returns = {}
    for ticker in tickers:
        stock_returns = ((portfolio_data[ticker] / portfolio_data[ticker].shift(1))-1).dropna().tolist()
        returns[ticker] = stock_returns

        single_index_reg = Ridge()
        single_index_reg.fit(factor_returns, stock_returns)
        stock_future = single_index_reg.predict(future_factors)
        future_returns[ticker] = stock_future

    future_returns = pd.DataFrame(future_returns)
    S = risk_models.sample_cov(future_returns)

    ef = EfficientFrontier(future_returns.mean(), S)
    weights = ef.max_sharpe(risk_free_rate=0.0)
    cleaned_weights = ef.clean_weights()
    print(cleaned_weights)

    ret, volatility, sharpe_ratio = ef.portfolio_performance()
    print("Expected annual return:", ret)
    print("Annual volatility:", volatility)
    print("Sharpe ratio:", sharpe_ratio)



----------------------------------------------------
Non-linear ML model: RandomForestRegressor()
[*********************100%***********************]  3 of 3 completed


/opt/homebrew/lib/python3.10/site-packages/pypfopt/risk_models.py:70: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(


OrderedDict([('AAPL', 0.0), ('AMZN', 0.00232), ('BRK-B', 0.0), ('GOOGL', 0.0), ('META', 0.0), ('MSFT', 0.0), ('NVDA', 0.78935), ('TSLA', 0.20833), ('UNH', 0.0), ('XOM', 0.0)])
Expected annual return: 0.0018754959302084834
Annual volatility: 1.649533452626864
Sharpe ratio: 0.0011369856896335001
----------------------------------------------------
Non-linear ML model: GradientBoostingRegressor()
[*********************100%***********************]  3 of 3 completed


/opt/homebrew/lib/python3.10/site-packages/pypfopt/risk_models.py:70: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/pypfopt/risk_models.py:89: UserWarning: Could not fix matrix. Please try a different risk model.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(


OrderedDict([('AAPL', 0.0), ('AMZN', 0.00609), ('BRK-B', 0.0), ('GOOGL', 0.0), ('META', 0.0), ('MSFT', 0.0), ('NVDA', 0.5438), ('TSLA', 0.28782), ('UNH', 0.00093), ('XOM', 0.16136)])
Expected annual return: 0.001515657627297046
Annual volatility: 0.20376655845670819
Sharpe ratio: 0.007438205948887631
----------------------------------------------------
Non-linear ML model: MLPRegressor(hidden_layer_sizes=(100, 100))
[*********************100%***********************]  3 of 3 completed
OrderedDict([('AAPL', 0.0), ('AMZN', 0.0), ('BRK-B', 0.0), ('GOOGL', 0.0), ('META', 0.0), ('MSFT', 0.0), ('NVDA', 0.3113), ('TSLA', 0.32039), ('UNH', 0.00291), ('XOM', 0.3654)])
Expected annual return: 0.0012702323830164623
Annual volatility: 0.27361679632411007
Sharpe ratio: 0.004642377222748494
----------------------------------------------------
Non-linear ML model: SVR()


/opt/homebrew/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(


[*********************100%***********************]  3 of 3 completed
OrderedDict([('AAPL', 0.0), ('AMZN', 0.0), ('BRK-B', 0.0), ('GOOGL', 0.0), ('META', 0.0), ('MSFT', 0.0), ('NVDA', 0.69851), ('TSLA', 0.27492), ('UNH', 0.0), ('XOM', 0.02658)])
Expected annual return: 0.0014966892173548416
Annual volatility: 0.5992097011586065
Sharpe ratio: 0.0024977720061289174


/opt/homebrew/lib/python3.10/site-packages/pypfopt/risk_models.py:70: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(


## Black Litterman model 

In [15]:
reg_models = [RandomForestRegressor(), GradientBoostingRegressor(), MLPRegressor(hidden_layer_sizes=(100, 100)), SVR()]

for reg_model in reg_models:
    print("----------------------------------------------------")
    print(f"Non-linear regression model: {reg_model}")
    
    # Using a non-linear ML model to predict stock prices in future
    returns = {}
    future_returns = {}
    for ticker in tickers:
        stock_returns = ((portfolio_data[ticker] / portfolio_data[ticker].shift(1))-1).dropna().tolist()
        returns[ticker] = stock_returns
        future_returns[ticker] = forecast_price(stock_returns, reg_model) 

    # Using the predicted future return of stocks as "views" for black litterman model
    viewdict = {key: value[-1] for key, value in future_returns.items()}
    print(f"Views: {viewdict}")

    # Obtain market caps for all stocks in portfolio
    mcaps = {}
    for t in tickers:
        stock = yf.Ticker(t)
        mcaps[t] = stock.info["marketCap"]

    # Determine prior estimate of returns implied by the market weights
    S = risk_models.CovarianceShrinkage(portfolio_data).ledoit_wolf()
    market_prices = yf.download("SPY", period="max")["Adj Close"]
    delta = black_litterman.market_implied_risk_aversion(market_prices)
    market_prior = black_litterman.market_implied_prior_returns(mcaps, delta, S)

    # Assign confidence measure for stock returns based on some heuristics
    confidences = [
        0.8,
        0.6,
        0.7,
        0.5,
        0.5,
        0.7,
        0.3,
        0.3,
        0.4,
        0.7,
        0.8,
        0.5,
        0.8,
        0.8,
        0.5
    ]

    # Fit the Black litterman model and calculate corresponding cov matrix and returns
    bl = BlackLittermanModel(S, pi=market_prior, absolute_views=viewdict, omega="idzorek", view_confidences=confidences)
    ret_bl = bl.bl_returns()
    S_bl = bl.bl_cov()

    # Find the optimal weights using bl cov matrix and returns
    ef = EfficientFrontier(ret_bl, S_bl)
    ef.add_objective(objective_functions.L2_reg)
    ef.max_sharpe()
    weights = ef.clean_weights()
    print(weights)

    ret, volatility, sharpe_ratio = ef.portfolio_performance()
    print("Expected annual return:", ret)
    print("Annual volatility:", volatility)
    print("Sharpe ratio:", sharpe_ratio)

----------------------------------------------------
Non-linear regression model: RandomForestRegressor()
Views: {'AAPL': 0.005511098031458916, 'AMZN': 0.00034489218142447317, 'BRK-B': -0.001068012150730886, 'GOOGL': -0.0031692898354079803, 'META': 0.006420880037666343, 'MSFT': -0.00271099173159011, 'NVDA': 0.007131304303162337, 'TSLA': 0.00868418058083816, 'UNH': 0.004397904375108709, 'XOM': 0.0031635761038197297, 'JPM': 0.010954975803260502, 'JNJ': 0.0024384318094717227, 'V': -0.003777244533047199, 'LLY': 0.0018822149111028409, 'PG': 0.0005840524215791843}
[*********************100%***********************]  1 of 1 completed
OrderedDict([('AAPL', 0.0), ('AMZN', 0.0), ('BRK-B', 0.0), ('GOOGL', 0.0), ('JNJ', 0.0), ('JPM', 0.0), ('LLY', 0.0), ('META', 0.02372), ('MSFT', 0.0), ('NVDA', 0.30813), ('PG', 0.0), ('TSLA', 0.66815), ('UNH', 0.0), ('V', 0.0), ('XOM', 0.0)])
Expected annual return: 0.05438473529159075
Annual volatility: 0.5508614481177361
Sharpe ratio: 0.062419934103360365
------

/opt/homebrew/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:259: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(


Views: {'AAPL': 0.003369621968943532, 'AMZN': 0.0018779775464757217, 'BRK-B': -0.0012471009025219752, 'GOOGL': -0.003588571422636571, 'META': 0.004890396967907461, 'MSFT': -0.0005347419523557852, 'NVDA': 0.007843136497233961, 'TSLA': -0.0018200927555136932, 'UNH': 0.0033411751101986914, 'XOM': -0.00018105640127292317, 'JPM': 0.0011939920582235944, 'JNJ': 0.0005918638294351189, 'V': -0.0008562308609232687, 'LLY': -0.0009728375461369773, 'PG': -0.0012073066043709864}
[*********************100%***********************]  1 of 1 completed
OrderedDict([('AAPL', 0.0), ('AMZN', 0.00719), ('BRK-B', 0.0), ('GOOGL', 0.0), ('JNJ', 0.0), ('JPM', 0.0), ('LLY', 0.0), ('META', 0.01573), ('MSFT', 0.0), ('NVDA', 0.3302), ('PG', 0.0), ('TSLA', 0.64688), ('UNH', 0.0), ('V', 0.0), ('XOM', 0.0)])
Expected annual return: 0.05236759023881758
Annual volatility: 0.5453112625674705
Sharpe ratio: 0.059356174098481575
----------------------------------------------------
Non-linear regression model: MLPRegressor(hid

/opt/homebrew/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:259: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(


Views: {'AAPL': 0.003674507947840147, 'AMZN': 0.0030936049324002957, 'BRK-B': 0.001422636568918112, 'GOOGL': 0.001686293244276913, 'META': 0.0038948461948367236, 'MSFT': 0.0020716085733893663, 'NVDA': 0.0066299660363618226, 'TSLA': -0.006771186339758539, 'UNH': 0.0037678376337792092, 'XOM': -0.0002035252537313792, 'JPM': -0.001372422209211313, 'JNJ': -0.0023598633859502616, 'V': -0.0022899466322585382, 'LLY': 0.00038182239173635557, 'PG': -0.00015262145118627446}
[*********************100%***********************]  1 of 1 completed
OrderedDict([('AAPL', 0.0), ('AMZN', 0.03343), ('BRK-B', 0.0), ('GOOGL', 0.0), ('JNJ', 0.0), ('JPM', 0.0), ('LLY', 0.0), ('META', 0.0256), ('MSFT', 0.0), ('NVDA', 0.33549), ('PG', 0.0), ('TSLA', 0.60548), ('UNH', 0.0), ('V', 0.0), ('XOM', 0.0)])
Expected annual return: 0.050873930319289246
Annual volatility: 0.5282659622438046
Sharpe ratio: 0.05844391372132424
----------------------------------------------------
Non-linear regression model: SVR()
Views: {'AAP

/opt/homebrew/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:259: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed
OrderedDict([('AAPL', 0.0), ('AMZN', 0.08023), ('BRK-B', 0.0), ('GOOGL', 0.0), ('JNJ', 0.0), ('JPM', 0.0), ('LLY', 0.0), ('META', 0.21625), ('MSFT', 0.0), ('NVDA', 0.27525), ('PG', 0.0), ('TSLA', 0.42827), ('UNH', 0.0), ('V', 0.0), ('XOM', 0.0)])
Expected annual return: 0.05397066389595893
Annual volatility: 0.4518205661514807
Sharpe ratio: 0.07518618327915971


/opt/homebrew/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:259: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(
